Everything is stolen from "https://github.com/pyg-team/pytorch_geometric/blob/master/examples/autoencoder.py"

Apparently it's only good for link prediction and thus it cannot predict node attributes. Specifically 
unuseful for attributemasking

In [1]:
import torch

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GAE, VGAE, GCNConv
import os
from mask import EdgeMask, AttributeMask

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cpu


In [ ]:
"""Preprocessed data """
dataset_dir = os.path.join(os.path.pardir, 'data/preprocessed/')
print(dataset_dir)
file_path=os.path.join(dataset_dir, 'meshgraphnets_miniset30traj5ts_vis.pt')
print(file_path)
dataset_full_timesteps = torch.load(file_path)
dataset = torch.load(file_path)[:1]
data2 = torch.load(file_path)[1:2]
print(len(dataset_full_timesteps)/5)

..\data/preprocessed/
..\data/preprocessed/meshgraphnets_miniset30traj5ts_vis.pt
30.0


*WITH EDGEMASKING*

In [7]:
transform = T.Compose([
    # Row-normalizes the attricutes to sum up to one
    T.NormalizeFeatures(),
    T.ToDevice(device),
    EdgeMask(p = 0.2),
    T.RandomLinkSplit(num_val=.05, num_test=.1, is_undirected=True,
                      # if split_labels is to true it will split positive and negative labels,
                      # and save them in distinct attributes
                      split_labels=True, 
                      # Add_negative_train_samples: Whether to add negative training samples for link
                      # prediction for link prediction. negative train samples might e.g. be edges that
                      # are not suposed to be in the graph.
                      add_negative_train_samples=False)
])

# dataset = Planetoid("\..", "CiteSeer", transform=transform)
pls = transform(dataset)
train_data, val_data, test_data = pls[0]

in_channels, out_channels = train_data.num_features, 16

*ATTRIBUTEMASKING*

In [3]:
transform = T.Compose([
    # Row-normalizes the attricutes to sum up to one
    T.NormalizeFeatures(),
    T.ToDevice(device),
    AttributeMask(p = 0.2)
])

pls = transform(dataset)
x, y = pls[0]

in_channels, out_channels = train_data.num_features, 16

In [12]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        # Mu is the difference between the GCNE and the VGCNE
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [13]:
variation = "GAE"
if variation == "GAE":
    model = GAE(GCNEncoder(in_channels, out_channels))
elif variation == "VGAE":
    model = VGAE(VariationalGCNEncoder(in_channels, out_channels))
else:
    raise Exception("Model type not specified")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

**TRAINING WITH EDGEMASK**

In [8]:
from torch.nn import CrossEntropyLoss
from torch_geometric.utils import to_dense_adj
cel = CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    target = to_dense_adj(train_data.edge_index)
    pred = model.decoder.forward_all(z) # Decodes z into a probabilistic dense adjacency matrix
                                        # Could also choose to decode the latent variable into edge probabilities for some given node pairs
    output = cel(pred.flatten(), target.squeeze().flatten())

    if variation == "VGAE":
        loss = loss + (1 / train_data.num_nodes) * model.kl_loss()

    output.backward()
    optimizer.step()

    # Can we remove float?
    return float(output)

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

In [19]:
import time
times = []
epochs = 100
for epoch in range(1, epochs + 1):
    start = time.time()
    output = train()
    auc, ap = test(test_data)
    print(f'Epoch: {epoch:03d}, AUC: {auc:.4f}, AP: {ap:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f})s")

tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9993, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
       grad_fn=<SigmoidBackward0>)
Epoch: 001, AUC: 0.6259, AP: 0.6044
tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9995, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
       grad_fn=<SigmoidBackward0>)
Epoch: 002, AUC: 0.6256, AP: 0.6046
tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]

**TRAINING WITH ATTRIBUTEMASK**

In [ ]:
from torch.nn import CrossEntropyLoss
from torch_geometric.utils import to_dense_adj
cel = CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x.x, x.edge_index)
    target = y
    pred = model.decoder.forward_all(z) # Decodes z into a probabilistic dense adjacency matrix
                                        # Could also choose to decode the latent variable into edge probabilities for some given node pairs
    output = cel(pred.flatten(), target.squeeze().flatten())
    # Only relevant if we use the variational graph auto encoder
    if variation == "VGAE":
        loss = loss + (1 / train_data.num_nodes) * model.kl_loss()
    


    output.backward()
    optimizer.step()

    # Can we remove float?
    return float(output)

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

In [9]:
from torch.utils.tensorboard import SummaryWriter

In [10]:
out_channels, num_features = 2, train_data.num_features
epochs = 100

model = GAE(GCNEncoder(in_channels, out_channels))
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [11]:
writer = SummaryWriter("runs/GAE_experiment_'+'2d_100_epochs")

In [12]:
import time
times = []
epochs = 100
for epoch in range(1, epochs + 1):
    output = train()
    auc, ap = test(test_data)
    writer.add_scalar('auc train', auc, epoch)
    writer.add_scalar('ap train', ap, epoch)